In [1]:
import pandas as pd
import numpy as np
import pylab as pl
import os 
import sys
import geopandas as gpd
from geopandas import GeoDataFrame
from shapely.geometry import Point
import scipy.stats as st
import statsmodels.formula.api as smf
from IPython.display import display, clear_output
from matplotlib.colors import ListedColormap
import time

%matplotlib inline

# Safety by School Data

In [2]:
SafetyQ12016 = pd.read_csv("https://www1.nyc.gov/assets/nypd/downloads/excel/school_safety/ssa-report-school-1q-2016.csv")
SafetyQ22016 = pd.read_csv("https://www1.nyc.gov/assets/nypd/downloads/excel/school_safety/2nd_qtr_2016_ssa_report_by_school_2016_09_28.csv")
SafetyQ32016 = pd.read_csv("https://www1.nyc.gov/assets/nypd/downloads/excel/school_safety/ssa-report-school-3q-2016.csv")
SafetyQ42016 = pd.read_csv("https://www1.nyc.gov/assets/nypd/downloads/excel/school_safety/ssa-report-school-4q-2016.csv")

SafetyQ12017 = pd.read_csv("https://www1.nyc.gov/assets/nypd/downloads/excel/school_safety/ssa-report-school-1q-2017.csv")
SafetyQ22017 = pd.read_csv("https://www1.nyc.gov/assets/nypd/downloads/excel/school_safety/ssa-report-school-2q-2017.csv")
SafetyQ32017 = pd.read_excel("https://www1.nyc.gov/assets/nypd/downloads/excel/school_safety/ssa-report-school-3q-2017.xlsx")



In [3]:
SafetyQ12017

,School Campus,Intervention Type,Incident Location,Greator Than 21,Scanning Type,Scanning Incident,School Related,School Hours,Enforcment By,Restraints,Force,Type Of Force,Top Charge,SummonsCharge,Classification
0,47 THE AMERICAN SIGN LANGUAGE & ENGLISH SECOND...,Child in Crisis,On-Site,No,Full-Time,N,Y,Y,NaN,No Restraints,N,None,NaN,NaN,NaN
1,Abraham Lincoln HS(BS),Arrested,Off-Site,No,Full-Time,N,N,N,PATROL,Metal,N,None,ROBBERY,NaN,F
2,Abraham Lincoln HS(BS),Arrested,Off-Site,No,Full-Time,N,N,N,PATROL,Metal,N,None,ROBBERY,NaN,F
3,Abraham Lincoln HS(BS),Arrested,Off-Site,Yes,Full-Time,N,N,N,PATROL,Metal,N,None,ASSAULT,NaN,M
4,Abraham Lincoln HS(BS),Arrested,Off-Site,No,Full-Time,N,N,Y,DB,No Restraints,N,None,ROBBERY,NaN,F
5,Abraham Lincoln HS(BS),Child in Crisis,On-Site,No,Full-Time,N,Y,Y,SSA,Metal,N,None,NaN,NaN,NaN
6,Abraham Lincoln HS(BS),Child in Crisis,On-Site,No,Full-Time,N,N,Y,NaN,No Restraints,N,None,NaN,NaN,NaN
7,Abraham Lincoln HS(BS),Juvenile Report,On-Site,No,Full-Time,Y,Y,Y,PATROL,No Restraints,N,None,NaN,NaN,Unk
8,Abraham Lincoln HS(BS),Mitigated,On-Site,No,Full-Time,N,Y,Y,DOE,No Restraints,N,None,NaN,NaN,NaN
9,Abraham Lincoln HS(BS),PINS / Warrant,Off-Site,No,Full-Time,N,Y,Y,PATROL,No Restraints,N,None,NaN,NaN,NaN


In [4]:
# Make sure Each Data set has same names
SafetyQ12016.rename(columns={' Arrest Top Charge':'TopCharge'}, inplace=True)
SafetyQ22016.rename(columns={'Arrest Top Charge':'TopCharge'}, inplace=True)
SafetyQ32016.rename(columns={'Top Charge':'TopCharge'}, inplace=True)
SafetyQ42016.rename(columns={'Top Charge':'TopCharge'}, inplace=True)
SafetyQ12017.rename(columns={'Top Charge':'TopCharge'}, inplace=True)
SafetyQ22017.rename(columns={'Top Charge':'TopCharge'}, inplace=True)
SafetyQ32017.rename(columns={'Top Charge':'TopCharge'}, inplace=True)


In [5]:
# Group Data into neat columns
SafetyGroupQ12016 = SafetyQ12016.groupby(('School Campus','Intervention Type'))['Incident Location'].count().reset_index()
SafetyGroupQ22016 = SafetyQ22016.groupby(('School Campus','Intervention Type'))['Incident Location'].count().reset_index()
SafetyGroupQ32016 = SafetyQ32016.groupby(('School Campus','Intervention Type'))['Incident Location'].count().reset_index()
SafetyGroupQ42016 = SafetyQ42016.groupby(('School Campus','Intervention Type'))['Incident Location'].count().reset_index()
SafetyGroupQ12017 = SafetyQ12017.groupby(('School Campus','Intervention Type'))['Incident Location'].count().reset_index()
SafetyGroupQ22017 = SafetyQ22017.groupby(('School Campus','Intervention Type'))['Incident Location'].count().reset_index()
SafetyGroupQ32017 = SafetyQ32017.groupby(('School Campus','Intervention Type'))['Incident Location'].count().reset_index()

# Merge Data Sets for years
SafetyGroup2016 = pd.concat([SafetyGroupQ12016, SafetyGroupQ22016, SafetyGroupQ32016, SafetyGroupQ42016]).reset_index()
SafetyGroup2017 = pd.concat([SafetyGroupQ12017, SafetyGroupQ22017, SafetyGroupQ32017]).reset_index()

# Regroup
SafetyGroup2016 = SafetyGroup2016.groupby(('School Campus','Intervention Type'))['Incident Location'].sum().reset_index()
SafetyGroup2017 = SafetyGroup2017.groupby(('School Campus','Intervention Type'))['Incident Location'].sum().reset_index()




In [6]:
# Pivot Data by intervention Type
SafetyPivot2016 = SafetyGroup2016.pivot(index='School Campus', columns='Intervention Type', values='Incident Location')
SafetyPivot2017 = SafetyGroup2017.pivot(index='School Campus', columns='Intervention Type', values='Incident Location')

In [7]:
# Add Total Columns
SafetyPivot2016['Total'] = SafetyPivot2016.sum(axis=1).values
SafetyPivot2017['Total'] = SafetyPivot2017.sum(axis=1).values

In [8]:
SafetyPivot2017.head()

Intervention Type,Arrested,Child in Crisis,Juvenile Report,Mitigated,PINS / Warrant,Summons,mitigated,Total
School Campus,,,,,,,,
47 THE AMERICAN SIGN LANGUAGE & ENGLISH SECONDARY SCHOOL(MS),NaN,3.0,NaN,NaN,NaN,1.0,NaN,4.0
ACADEMY FOR SCHOLARSHIP & ENTREPRENEURSHIP(XE),NaN,1.0,1.0,NaN,NaN,NaN,NaN,2.0
ACADEMY OF ENVIRONMENTAL SCIENCE SECONDARY HS(MN),NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0
ACADEMY OF URBAN PLANNING(BN),NaN,NaN,1.0,12.0,NaN,NaN,1.0,14.0
ACORN COMMUNITY HS(BN),NaN,2.0,1.0,3.0,NaN,1.0,2.0,9.0


In [9]:
# Merge data sets, need to still be relabelled
pd.merge(SafetyPivot2016.reset_index(), 
         SafetyPivot2017.reset_index(), 
         on="School Campus", how="outer")

Intervention Type,School Campus,Arrested_x,Child in Crisis_x,Juvenile Report_x,Mitigated_x,PINS / Warrant_x,PINS Warrant,Summons_x,Total_x,Arrested_y,Child in Crisis_y,Juvenile Report_y,Mitigated_y,PINS / Warrant_y,Summons_y,mitigated,Total_y
0,47 THE AMERICAN SIGN LANGUAGE & ENGLISH SECOND...,NaN,NaN,1.0,1.0,NaN,NaN,NaN,2.0,NaN,3.0,NaN,NaN,NaN,1.0,NaN,4.0
1,ACADEMY OF AMERICAN STUDIES(QN),NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ACADEMY OF URBAN PLANNING(BN),NaN,NaN,NaN,2.0,1.0,NaN,3.0,6.0,NaN,NaN,1.0,12.0,NaN,NaN,1.0,14.0
3,ACHIEVEMENT FIRST CROWN HEIGHTS CHARTER SCHOOL...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ACORN COMMUNITY HS(BN),1.0,NaN,1.0,6.0,NaN,NaN,1.0,9.0,NaN,2.0,1.0,3.0,NaN,1.0,2.0,9.0
5,ACORN HS FOR SOCIAL JUSTICE(BN),NaN,NaN,1.0,NaN,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,ADLAI STEVENSON HS GED(XE),NaN,1.0,NaN,NaN,NaN,NaN,1.0,2.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0
7,ALC - August Martin HS@A. Martin HS (QS),1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,1.0,NaN,NaN,NaN,2.0
8,ALC-JAMAICA ACADEMY@ALC-Complex (Old PS 182Q -...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,2.0
9,ALC-Royal 180 Academy@ALC-Complex(Old PS182Q-Qns),1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
# Convert to Csv for processing
SafetyPivot2016.to_csv("data/SafetyStats2016.csv")
SafetyPivot2017.to_csv("data/SafetyStats2017.csv")

# Other Safety Number

In [10]:
# Create data folder
os.system("mkdir data")

# Download Data
!curl https://data.cityofnewyork.us/api/views/qybk-bjjc/rows.csv?accessType=DOWNLOAD > data/SafetyStats2010_2016.csv  
# No data before 2013
# link to data dictionary
# https://data.cityofnewyork.us/api/views/qybk-bjjc/files/d7ff6a2c-1706-48b3-922f-be57d4e066f7?download=true&filename=2015-16%20School%20Safety%20Report%20Data%20Dictionary.xlsx

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 2174k    0 2174k    0     0  2174k      0 --:--:-- --:--:-- --:--:-- 2318k
100 2471k    0 2471k    0     0  2471k      0 --:--:--  0:00:01 --:--:-- 2432k


In [11]:
# Read Data
Safety13_16 = pd.read_csv('data/SafetyStats2010_2016.csv')

# Check columns
Safety13_16.columns

Index([u'School Year', u'Building Code', u'DBN', u'Location Name',
       u'Location Code', u'Address', u'Borough', u'Geographical District Code',
       u'Register', u'Building Name', u'# Schools', u'Schools in Building',
       u'Major N', u'Oth N', u'NoCrim N', u'Prop N', u'Vio N', u'ENGroupA',
       u'RangeA', u'AvgOfMajor N', u'AvgOfOth N', u'AvgOfNoCrim N',
       u'AvgOfProp N', u'AvgOfVio N', u'Borough Name', u'Postcode',
       u'Latitude', u'Longitude', u'Community Board', u'Council District ',
       u'Census Tract', u'BIN', u'BBL', u'NTA'],
      dtype='object')

In [12]:
# Sort Data
Safety13_16 = Safety13_16[['School Year', 'Building Code', 'DBN', 
                           'Location Name','Location Code',
                           'Major N', 'Oth N', 'NoCrim N', 
                           'Prop N', 'Vio N']]

# CReate total column
Safety13_16['TotalCrimes'] = Safety13_16[['Major N','Oth N',
                                          'NoCrim N','Prop N',
                                          'Vio N']].sum(axis=1)

# Check unique years
Safety13_16['School Year'].unique()

array(['2013-14', '2014-15', '2015-16'], dtype=object)

In [13]:
Safety13_16.head()

,School Year,Building Code,DBN,Location Name,Location Code,Major N,Oth N,NoCrim N,Prop N,Vio N,TotalCrimes
0,2013-14,K001,15K001,P.S. 001 The Bergen,K001,0.0,2.0,1.0,1.0,0.0,4.0
1,2013-14,K002,17K002,Parkside Preparatory Academy,K002,NaN,NaN,NaN,NaN,NaN,0.0
2,2013-14,K002,75K141,P.S. K141,K141,NaN,NaN,NaN,NaN,NaN,0.0
3,2013-14,K002,84K704,Explore Charter School,K704,NaN,NaN,NaN,NaN,NaN,0.0
4,2013-14,K002,NaN,655 PARKSIDE AVENUE CONSOLIDATED LOCATION,NaN,1.0,5.0,2.0,2.0,4.0,14.0


In [90]:
Safety13_16.to_csv("data/SafetyStats2013-2016.csv")

In [16]:
# Download Data
!curl https://data.cityofnewyork.us/api/views/qgea-i56i/rows.csv?accessType=DOWNLOAD > data/NYPD_Complaint_Data_Historic.csv

The process cannot access the file because it is being used by another process.


In [17]:
Safety3 = pd.read_csv('data/NYPD_Complaint_Data_Historic.csv')

C:\Users\Andrew\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2718: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [19]:
Safety3.head()

,CMPLNT_NUM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,RPT_DT,KY_CD,OFNS_DESC,PD_CD,PD_DESC,...,ADDR_PCT_CD,LOC_OF_OCCUR_DESC,PREM_TYP_DESC,PARKS_NM,HADEVELOPT,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lat_Lon
0,667475431,12/30/2015,20:48:00,12/31/2015,11:29:00,12/31/2015,107,BURGLARY,213,"BURGLARY,COMMERCIAL,NIGHT",...,40,INSIDE,PRIVATE/PAROCHIAL SCHOOL,NaN,NaN,1006699.0,233128.0,40.806538,-73.918911,"(40.806538101, -73.918910596)"
1,938267555,12/29/2015,13:00:00,12/29/2015,13:01:00,12/30/2015,361,OFF. AGNST PUB ORD SENSBLTY &,639,AGGRAVATED HARASSMENT 2,...,112,INSIDE,PUBLIC SCHOOL,NaN,NaN,1025286.0,202643.0,40.722798,-73.851956,"(40.722797981, -73.851956357)"
2,178705935,12/28/2015,12:00:00,12/28/2015,18:30:00,12/29/2015,109,GRAND LARCENY,403,"LARCENY,GRAND BY BANK ACCT COMPROMISE-REPRODUC...",...,24,INSIDE,PUBLIC SCHOOL,NaN,NaN,992460.0,227653.0,40.791535,-73.970352,"(40.791535421, -73.970351765)"
3,601105377,12/22/2015,13:25:00,12/22/2015,13:25:00,12/29/2015,109,GRAND LARCENY,438,"LARCENY,GRAND FROM BUILDING (NON-RESIDENCE) UN...",...,104,INSIDE,PRIVATE/PAROCHIAL SCHOOL,NaN,NaN,1015418.0,198786.0,40.712252,-73.887575,"(40.71225172, -73.887574836)"
4,427817978,12/15/2015,14:35:00,NaN,NaN,12/22/2015,578,HARRASSMENT 2,638,"HARASSMENT,SUBD 3,4,5",...,20,INSIDE,PUBLIC SCHOOL,NaN,NaN,988503.0,221442.0,40.774491,-73.984646,"(40.774490663, -73.984646049)"


In [ ]:
# No schools identified, Could match with Location coords

# School Class Size
### Can we get this from attendance data?

In [56]:
ClassSize2018 = pd.read_excel("http://schools.nyc.gov/NR/rdonlyres/32A4D266-83DB-439D-AE73-EDD1BC23E9AD/0/February2018_Avg_ClassSize_School.xlsx",header=7)
ClassSize2017 = pd.read_excel("http://schools.nyc.gov/NR/rdonlyres/11A3268A-AA1B-4475-BA79-A9D52883DC08/0/February2017_Avg_ClassSize_School.xlsx",header=7)
ClassSize2016 = pd.read_excel("http://schools.nyc.gov/NR/rdonlyres/3B90A8E1-5B32-4B86-9BB3-4FCFC30A8182/0/February2016_Avg_ClassSize_School.xlsx",header=6)
ClassSize2015 = pd.read_excel("http://schools.nyc.gov/NR/rdonlyres/5EC03778-74AD-45FF-9951-44441AE570F2/0/SchoolLevelDetailSummary_Updated_2015.xlsx",header=10)
ClassSize2014 = pd.read_excel("http://schools.nyc.gov/offices/d_chanc_oper/budget/dbor/DBOR_CLASS_SIZE/FY14_Data/Updated_School_level_Detail_Summary_2014_02_13.xlsx",header=10)
ClassSize2013 = pd.read_excel("http://schools.nyc.gov/offices/d_chanc_oper/budget/dbor/DBOR_CLASS_SIZE/FY13_Data/20122013UpdatedSchoolLevelDetailFinal%202012_12_14.xlsx",header=10)
ClassSize2012 = pd.read_excel("http://schools.nyc.gov/offices/d_chanc_oper/budget/dbor/DBOR_CLASS_SIZE/FY12_Data/Updated_School_Level_Detail_21512.xls",header=10)
ClassSize2011 = pd.read_excel("http://schools.nyc.gov/NR/rdonlyres/5E4CE3DD-739E-4DEF-86CF-AEBFD87715BB/0/20102011PreliminarySchoolLevelDetail.xls",header=10)
ClassSize2010 = pd.read_excel("http://schools.nyc.gov/NR/rdonlyres/B887E47A-30EB-4D80-A5DA-46946B67C239/0/20102011UpdatedSchoolLevelDetail.xls",header=10)
# ClassSize2009 = pd.read_excel("")
# ClassSize2008 = pd.read_excel("")
# ClassSize2007 = pd.read_excel("")
# ClassSize2006 = pd.read_excel("")

In [57]:
list(ClassSize2018.DBN.unique())

[u'01M015',
 u'01M019',
 u'01M020',
 u'01M034',
 u'01M063',
 u'01M064',
 u'01M110',
 u'01M134',
 u'01M140',
 u'01M142',
 u'01M184',
 u'01M188',
 u'01M301',
 u'01M315',
 u'01M332',
 u'01M361',
 u'01M363',
 u'01M364',
 u'01M378',
 u'01M450',
 u'01M539',
 u'01M839',
 u'02M001',
 u'02M002',
 u'02M003',
 u'02M006',
 u'02M011',
 u'02M033',
 u'02M040',
 u'02M041',
 u'02M042',
 u'02M051',
 u'02M059',
 u'02M077',
 u'02M089',
 u'02M104',
 u'02M111',
 u'02M114',
 u'02M116',
 u'02M124',
 u'02M126',
 u'02M130',
 u'02M131',
 u'02M150',
 u'02M151',
 u'02M158',
 u'02M167',
 u'02M177',
 u'02M183',
 u'02M198',
 u'02M212',
 u'02M217',
 u'02M225',
 u'02M234',
 u'02M255',
 u'02M260',
 u'02M267',
 u'02M276',
 u'02M281',
 u'02M289',
 u'02M290',
 u'02M297',
 u'02M312',
 u'02M340',
 u'02M343',
 u'02M347',
 u'02M397',
 u'02M407',
 u'02M408',
 u'02M413',
 u'02M422',
 u'02M442',
 u'02M527',
 u'02M655',
 u'02M896',
 u'02M933',
 u'03M009',
 u'03M054',
 u'03M075',
 u'03M076',
 u'03M084',
 u'03M087',
 u'03M145',
 u'0

In [59]:
ClassSize2010.head()

,CSD,BOROUGH,SCHOOL CODE,SCHOOL NAME,GRADE,PROGRAM TYPE,CORE SUBJECT (MS CORE and 9-12 ONLY),CORE COURSE (MS CORE and 9-12 ONLY),SERVICE CATEGORY (K-9* ONLY),NUMBER OF STUDENTS / SEATS FILLED,NUMBER OF SECTIONS,AVERAGE CLASS SIZE,SIZE OF SMALLEST CLASS,SIZE OF LARGEST CLASS,DATA SOURCE,SCHOOLWIDE PUPIL-TEACHER RATIO
0,1,M,M015,P.S. 015 Roberto Clemente,0K,GEN ED,-,-,-,19.0,1.0,19.0,19.0,19.0,ATS,NaN
1,1,M,M015,P.S. 015 Roberto Clemente,0K,CTT,-,-,-,21.0,1.0,21.0,21.0,21.0,ATS,NaN
2,1,M,M015,P.S. 015 Roberto Clemente,01,GEN ED,-,-,-,17.0,1.0,17.0,17.0,17.0,ATS,NaN
3,1,M,M015,P.S. 015 Roberto Clemente,01,CTT,-,-,-,17.0,1.0,17.0,17.0,17.0,ATS,NaN
4,1,M,M015,P.S. 015 Roberto Clemente,02,GEN ED,-,-,-,15.0,1.0,15.0,15.0,15.0,ATS,NaN


# School Attendance

In [4]:
# Read in data on school attendance
Attend06_09 = pd.read_csv("https://data.cityofnewyork.us/api/views/xwxx-rnki/rows.csv?accessType=DOWNLOAD")
Attend09_12 = pd.read_csv("https://data.cityofnewyork.us/api/views/wpqj-3buw/rows.csv?accessType=DOWNLOAD")
Attend12_17 = pd.read_csv("https://data.cityofnewyork.us/api/views/wed3-5i35/rows.csv?accessType=DOWNLOAD")

#### Try and Visualise data and aggreagate into years

In [5]:
# Visualise and try to aggregate data into years
Attend06_09.head()

,School,Date,SchoolYear,Enrolled,Present,Absent,Released
0,01M015,20060905,20062007,252,226,26,0
1,01M015,20060906,20062007,248,227,21,0
2,01M015,20060907,20062007,245,228,17,0
3,01M015,20060908,20062007,246,226,20,0
4,01M015,20060911,20062007,248,223,25,0


In [14]:
# Visualise and try to aggregate data into years
Attend09_12.head()

,School,Date,SchoolYear,Enrolled,Present,Absent,Released
0,01M015,20090909,20092010,210,174,36,0
1,01M015,20090910,20092010,215,193,22,0
2,01M015,20090911,20092010,213,185,28,0
3,01M015,20090914,20092010,214,199,15,0
4,01M015,20090915,20092010,214,194,20,0


In [15]:
Attend12_17.head()

,School,MonthCode,CalMonth,SchoolYear,GradeLevel,GradeSort,RosterCount,Present,Absent,Released
0,14K257,4,Dec,2012 - 2013,0K,0,106,1456,126,0
1,14K257,4,Dec,2014 - 2015,PK,-1,66,974,148,0
2,14K257,4,Dec,2013 - 2014,PK,-1,53,677,65,53
3,14K257,4,Dec,2016 - 2017,PK,-1,58,860,95,0
4,14K257,4,Dec,2015 - 2016,PK,-1,61,881,95,61


### For 2012 to 2017 we can split into grades to get a finer view 

In [54]:
# Create data groups per year
Attend06_09group = Attend06_09.groupby(["School",'SchoolYear'])['Enrolled','Present','Absent','Released'].sum()
Attend09_12group = Attend09_12.groupby(["School",'SchoolYear'])['Enrolled','Present','Absent','Released'].sum()
Attend12_17group = Attend12_17.groupby(["School",'SchoolYear'])['RosterCount','Present','Absent','Released'].sum()

# Merge into 1 dataset
Attendance = pd.concat([Attend06_09group,Attend09_12group,Attend12_17group]).reset_index()

# Create Attendance Percentage for comparison
# Can we compare percentages? Should aggregate over a year right?
Attendance['Percent'] = (100.0 * Attendance.Present / 
                         (Attendance.Present + Attendance.Absent + 
                          Attendance.Released))

# Review data for specific school
Attendance[Attendance['School'] == "14K257"]

,School,SchoolYear,Absent,Enrolled,Present,Released,RosterCount,Percent
2068,14K257,20062007,7359,104379.0,96914,106,NaN,92.848178
2069,14K257,20072008,7079,103707.0,96628,0,NaN,93.174038
2070,14K257,20082009,7746,102562.0,94816,0,NaN,92.447495
6638,14K257,20092010,6243,101483.0,95240,0,NaN,93.848231
6639,14K257,20102011,7007,107728.0,100651,70,NaN,93.430677
6640,14K257,20112012,7313,115859.0,108460,86,NaN,93.613789
12888,14K257,2012 - 2013,6802,NaN,106579,1418,6465.0,92.839659
12889,14K257,2013 - 2014,8336,NaN,110871,917,6706.0,92.297126
12890,14K257,2014 - 2015,8238,NaN,114407,259,6845.0,93.086474
12891,14K257,2015 - 2016,7884,NaN,110530,61,6744.0,93.293944


In [52]:
Attendance.head()

,School,SchoolYear,Absent,Enrolled,Present,Released,RosterCount,Percent
0,01M015,20062007,4608,46255.0,41647,0,NaN,90.037834
1,01M015,20072008,4596,46543.0,41760,187,NaN,89.723482
2,01M015,20082009,5109,44377.0,39235,33,NaN,88.412917
3,01M019,20062007,5045,57738.0,52613,80,NaN,91.123697
4,01M019,20072008,5477,61613.0,56136,0,NaN,91.110642
